# Function calling for nearby places: Leveraging the Bing Maps Local Search api

This notebook is centered around the integration of the Bing Maps Local Search api to enhance location-based searches. Please note that while we focus on the Bing Maps Local Search api in this instance, there are numerous other APIs you could explore and apply in a similar fashion.

We'll explore the application of three main components:

- Bing Maps Local Search api: This API provides real-time data about nearby places. It factors in various data points such as ratings, types of venues, costs, and more from the locations around you.

- Function calling: A single command such as \"I'm hungry\" or \"I want to visit a museum\" activates the function which invokes the Bing Maps Local Search api to identify suitable venues.

This notebook introduces two primary use cases:

- API integration with function calling: Understand how to integrate and call Bing Maps Local Search api effectively to source real-time data of various places using function calling.

Please note that while this system is highly versatile, its effectiveness may vary based on user preferences and available place data. For the purposes of this notebook, the customer data is fake and the location is hardcoded. "

## Setup

Bing Maps Local Search api

To use the Bing Maps Local Search api, you'll need:

- Microsoft Account: If you don't already have one, you will need to create a Microsoft account.

- Bing Maps Local Search api API Key: The API key is a unique identifier that is used to authenticate requests associated with your project. You can get your API key from the [Bing Maps Dev Center](https://www.bingmapsportal.com/Application). 


## Installation
Install the Azure Open AI SDK using the below command.

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.9"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.9

In [2]:
#r "nuget:Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23552.1"

using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.AIUtilities;

Installed Packages Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23552.1

## Run this cell, it will prompt you for the apiKey, endPoint, gtpDeployment, and bingMap

In [3]:
var azureOpenAIKey = await Kernel.GetPasswordAsync("Provide your OPEN_AI_KEY");

// Your endpoint should look like the following https://YOUR_OPEN_AI_RESOURCE_NAME.openai.azure.com/
var azureOpenAIEndpoint = await Kernel.GetInputAsync("Provide the OPEN_AI_ENDPOINT");

// Enter the deployment name you chose when you deployed the model.
var gptDeployment = await Kernel.GetInputAsync("Provide GPT  deployment name");

var bingMapsKey = await Kernel.GetPasswordAsync("Provide your BING_MAPS_KEY");

### Import namesapaces and create an instance of `OpenAiClient` using the `azureOpenAIEndpoint` and the `azureOpenAIKey`

In [4]:
using Azure;
using Azure.AI.OpenAI;

In [5]:
OpenAIClient client = new (new Uri(azureOpenAIEndpoint), new AzureKeyCredential(azureOpenAIKey.GetClearTextPassword()));

In [6]:
public enum PlaceType{
    Bars,
    BarsGrillsAndPubs,
    BelgianRestaurants,
    BreweriesAndBrewPubs,
    BritishRestaurants,
    BuffetRestaurants,
    CafeRestaurants,
    CaribbeanRestaurants,
    ChineseRestaurants,
    CocktailLounges,
    CoffeeAndTea,
    Delicatessens,
    DeliveryService,
    Diners,
    DiscountStores,
    Donuts,
    FastFood,
    FrenchRestaurants,
    FrozenYogurt,
    GermanRestaurants,
    GreekRestaurants,
    Grocers,
    Grocery,
    HawaiianRestaurants,
    HungarianRestaurants,
    IceCreamAndFrozenDesserts,
    IndianRestaurants,
    ItalianRestaurants,
    JapaneseRestaurants,
    Juices,
    KoreanRestaurants,
    LiquorStores,
    MexicanRestaurants,
    MiddleEasternRestaurants,
    Pizza,
    PolishRestaurants,
    PortugueseRestaurants,
    Pretzels,
    Restaurants,
    RussianAndUkrainianRestaurants,
    Sandwiches,
    SeafoodRestaurants,
    SpanishRestaurants,
    SportsBars,
    SteakHouseRestaurants,
    Supermarkets,
    SushiRestaurants,
    TakeAway,
    Taverns,
    ThaiRestaurants,
    TurkishRestaurants,
    VegetarianAndVeganRestaurants,
    VietnameseRestaurants,
    AmusementParks,
    Attractions,
    Carnivals,
    Casinos,
    LandmarksAndHistoricalSites,
    MiniatureGolfCourses,
    MovieTheaters,
    Museums,
    Parks,
    SightseeingTours,
    TouristInformation,
    Zoos,
    AntiqueStores,
    Bookstores,
    CDAndRecordStores,
    ChildrensClothingStores,
    CigarAndTobaccoShops,
    ComicBookStores,
    DepartmentStores,
    FleaMarketsAndBazaars,
    FurnitureStores,
    HomeImprovementStores,
    JewelryAndWatchesStores,
    KitchenwareStores,
    MallsAndShoppingCenters,
    MensClothingStores,
    MusicStores,
    OutletStores,
    PetShops,
    PetSupplyStores,
    SchoolAndOfficeSupplyStores,
    ShoeStores,
    SportingGoodsStores,
    ToyAndGameStores,
    VitaminAndSupplementStores,
    WomensClothingStores,
    BanksAndCreditUnions,
    Hospitals,
    HotelsAndMotels,
    Parking
}

This ustility functions helps translating a `GptFunction` to the `FunctionDefinition` type

In [8]:
using System.Text.Json;

public FunctionDefinition CreateFunctionDefinition(GptFunction function)
{
    var functionDefinition = new FunctionDefinition(function.Name);
    var json = JsonDocument.Parse(function.JsonSignature.ToString()).RootElement;
    functionDefinition.Parameters = BinaryData.FromString(json.GetProperty("parameters").ToString());
    return functionDefinition;
}

## Create a `GptFunction`

A `GptFunction` is an object  that can be used to create a `ChatCompletionOption` and later execute the logic in the `delegate` passed in the constructor.

Let's create ones that will take as input a location and a placeType. They will be used to search a matching location using the [Bing Map Local Search Api](https://learn.microsoft.com/bingmaps/rest-services/locations/local-search).

In [7]:
using System.Web;
var findFunction = GptFunction.Create("find",async (string currentLocation, PlaceType placeType) =>{
    var apiKey = bingMapsKey.GetClearTextPassword();
    var httpClient = new System.Net.Http.HttpClient();
    var url = new Uri($"https://dev.virtualearth.net/REST/v1/LocalSearch/?query={HttpUtility.UrlEncode(currentLocation)}&type={placeType}&key={apiKey}");
 
    var response = await httpClient.GetAsync(url);
    return await response.Content.ReadAsStringAsync();   
},enumsAsString: true);


### Create ProvideReccomendations
This function will use the `GptFunction` and `ChatGPT` to answer a user question by calling the function with parameters generated by the LLM.


In [12]:
public async Task<string> ProvideReccomendations(string userQuestion){
    var response = await client.GetChatCompletionsAsync(new ChatCompletionsOptions{
        Messages={
                    new ChatMessage(ChatRole.System, "You are a sophisticated AI assistant, a specialist in user intent detection and interpretation. Your task is to perceive and respond to the user's needs, even when they're expressed in an indirect or direct manner. You excel in recognizing subtle cues: for example, if a user states they are 'hungry', you should assume they are seeking nearby dining options such as a restaurant or a cafe. If they indicate feeling 'tired', 'weary', or mention a long journey, interpret this as a request for accommodation options like hotels or guest houses. However, remember to navigate the fine line of interpretation and assumption: if a user's intent is unclear or can be interpreted in multiple ways, do not hesitate to politely ask for additional clarification. Use only values from the nums in the functions."),
                    new ChatMessage(ChatRole.User, userQuestion)
        },
        Functions = new []{ CreateFunctionDefinition(findFunction) },
        DeploymentName = gptDeployment,
    });

    var functionCall =  response.Value.Choices[0].Message.FunctionCall;
    
    var results = await  ((Task<string>) findFunction.Execute(functionCall.Arguments));
    return results;
}

In [14]:
var response = await ProvideReccomendations("I am hungry in Seattle, actually Capitol Hill. At the moment I would appreaciate something local and cheap. Maybe a pub? Don't know what is the best to go for. I am open to any idea, What do you suggest?");

response.Display();

{"authenticationResultCode":"ValidCredentials","brandLogoUri":"https:\/\/dev.virtualearth.net\/Branding\/logo_powered_by.png","copyright":"Copyright © 2023 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.","resourceSets":[{"estimatedTotal":0,"resources":[]}],"statusCode":200,"statusDescription":"OK","traceId":"130bb839dcb6418486ce0fda8b2b40ab|DU0000305A|0.0.0.0|Ref A: 94D86322B6E3481E8D821AAB8D823EBF Ref B: DB3EDGE2219 Ref C: 2023-11-07T12:47:44Z"}